In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pymorphy2[fast]

     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 8.2 MB 10.3 MB/s 
     |████████████████████████████████| 371 kB 53.0 MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=849485 sha256=dbc158d980bd436dec93d1f8e25c357d72ac61428702ecd80ee174d1b0745af0
  Stored in directory: /root/.cache/pip/wheels/85/51/a4/2de41ff197786537075027c27b479a38da92f50abc86634445
Successfully built DAWG


In [3]:
import random
import re
import string
import pandas as pd
import torch
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import gensim
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchvision.transforms import ToTensor
import warnings 
from sklearn.linear_model import LogisticRegression
from PIL import Image
from IPython.display import clear_output
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from typing import List
from sklearn.metrics import classification_report
import pickle
from typing import List
import nltk
import string
import pymorphy2
import codecs

In [4]:
warnings.filterwarnings(action='ignore',category=UserWarning, module='gensim')  
warnings.filterwarnings(action='ignore',category=FutureWarning, module='gensim')  
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='gensim')
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='smart_open') 
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='sklearn')
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='scipy')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [13]:
class Preparator():
  def __init__(self):
      self.morph = pymorphy2.MorphAnalyzer()
      self.tokenizer = nltk.WordPunctTokenizer()
      self.stopwords = set(line.strip() for line in codecs.open('/content/drive/MyDrive/aiijc_sber/all_stopwords.txt', "r", "utf_8_sig").readlines())
      self.bigram_mod = gensim.models.Phrases.load("/content/drive/MyDrive/aiijc_sber/bigram_model.pkl")

  def prepare_corp(self, news_list: List[str]):
      return [self.newstext2token(news_text) for news_text in news_list]

  def make_bigrams(self, doc):
      return self.bigram_mod[doc]

  def newstext2token(self, news_text: str):
      tokens = self.tokenizer.tokenize(news_text.lower())
      tokens_with_no_punct = [self.morph.parse(w)[0].normal_form for w in tokens if all(c in 'йцукенгшщзхъёфывапролджэячсмитьбю' for c in w)]
      tokens_base_forms = [w for w in tokens_with_no_punct if w not in self.stopwords]
      tokens_last = [w for w in tokens_base_forms if len(w)>1]
      tokens_bigrammed = self.make_bigrams(tokens_last)
      return tokens_bigrammed

  

In [14]:
print(Preparator().newstext2token('Глобус - модель земного шара'))

['глобус', 'модель', 'земной_шар']
